In [2]:
import pandas as pd
import numpy as np 
from pandas.api.types import CategoricalDtype
from datetime import datetime 
import pandas as pd
import numpy as np 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix



Load the cleaned dataset which contains 6k rows of movie info:

In [3]:
movies = pd.read_csv("Data/movies.csv")

In [4]:
movies.shape

(6020, 33)

### Searching the dataframe for the movies which have similar name to the one that is searched

- vectorizing the `title` 
- fit and transform the titles in the tfidf vectorizer.

In [5]:
#Define a TF-IDF Vectorizer Object for titles
tfidf_title = TfidfVectorizer(stop_words='english',min_df=0,ngram_range=(1,4))


#Construct the required TF-IDF matrix by fitting and transforming the data
title_matrix = tfidf_title.fit_transform(movies['title'])

#Output the shape of tfidf_matrix
title_matrix.shape

(6020, 12139)

- Getting the cosine similarity of the title 

In [6]:
title_similarity = cosine_similarity(title_matrix)


In [7]:
title_similarity.shape
#sanity check on the shape of our matrix

(6020, 6020)

#### `Content_recommender` function 
which gets a title for a movie and returns the top movies with similar name to the one searched.


In [8]:
# def content_recommender(title, movies,similarities) :
    
#     idx = movies[movies['title']==title].index
#     if  idx.empty:
#         matching_movies = movies[movies['title'].str.contains(title, case=False)]
#         matching_indices = matching_movies.index
#         idx = matching_indices[0]
#         if matching_movies.empty:
#             raise ValueError("No similar movie titles found.")
        
    

    
# # Create a dataframe with the movie titles
#     sim_df = pd.DataFrame({'movie':movies['title'], 
#                        'similarity': title_similarity[idx]})
#     # Get the top 10 movies with 
#     top_movies = sim_df.sort_values(by='similarity', ascending=False).head(10)

#     return top_movies

In [51]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

def find_similar_movies(movie_title, top_n=10, threshold=80):
    # Calculate similarity scores for each movie title
    titles = movies['title'].tolist()
    similarity_scores = process.extract(movie_title, titles, scorer=fuzz.partial_ratio)

    # Filter movies with similarity scores above the threshold
    
    similar_movies = [name for name,score in similarity_scores if score >= threshold]
    # print(similar_movies[:top_n])
    return similar_movies[:top_n]



In [52]:

similar_movies = find_similar_movies("batman", top_n=10, threshold=80)
print(similar_movies)

['Batman Forever', 'Batman', 'M', 'Batman Returns', 'Batman & Robin']


- Creating a tfidf vectorizer
- fit and transform 5000 most repeated woreds in the movie reviews

In [10]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',min_df=10,ngram_range=(1,2),max_features=5000)


#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(6020, 3269)

Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [11]:
overview_similarity = cosine_similarity(tfidf_matrix)

### `Recommendr_on_story` 

In [12]:
def recommender_on_story(title,similarities):
    idx = movies[movies['title']==title].index
    if  idx.empty:
        matching_movies = movies[movies['title'].str.contains(title, case=False)]
        matching_indices = matching_movies.index
        idx = matching_indices[0]
        if matching_movies.empty:
            raise ValueError("No similar movie titles found.")
    
    sim_df = pd.DataFrame({'movie':movies['title'], 
                       'similarity': similarities[idx]})
    top_movies = sim_df.sort_values(by='similarity', ascending=False).head(10)
    return top_movies

In [21]:
recommender_on_story("mad max",overview_similarity)

,movie,similarity
1413,Mad Max,1.000000
2782,District B13,0.186573
3370,Australia,0.183906
4240,Texas Killing Fields,0.183822
3575,Surrogates,0.180333
1970,Black Christmas,0.179205
4579,New World,0.178068
5781,The Light Between Oceans,0.174850
6007,Banana,0.174773
467,The Sting,0.169804


First:

Load the data farme that contains genres dummies and movieIds.

In [14]:
genres_df = pd.read_csv("Data/genres_dummies.csv")
genres_df.drop("Unnamed: 0",axis=1,inplace=True)

Then we're going to define a cosine similarity matrix for them:

In [15]:
genre_columns = genres_df.columns[1:]

# Calculate the similarity matrix based on genres
genre_similariy = cosine_similarity(genres_df[genre_columns])

In [16]:
genre_similariy.shape

(6020, 6020)

In [17]:
movies.shape

(6020, 33)

In [18]:
def recommender_on_genre(title,similarities):
    idx = movies[movies['title']==title].index
    if  idx.empty:
        matching_movies = movies[movies['title'].str.contains(title, case=False)]
        matching_indices = matching_movies.index
        idx = matching_indices[0]
        if matching_movies.empty:
            raise ValueError("No similar movie titles found.")
    print(idx)    
    sim_df = pd.DataFrame({'movie':movies['title'], 
                       'similarity': similarities[idx]})
    top_movies = sim_df.sort_values(by='similarity', ascending=False).head(10)
    return top_movies

In [19]:
recommender_on_genre("Harry potter",genre_similariy)

1741


,movie,similarity
3859,The Chronicles of Narnia: The Voyage of the Da...,1.0
3849,Harry Potter and the Deathly Hallows: Part 1,1.0
3217,The Spiderwick Chronicles,1.0
980,Mighty Joe Young,1.0
5343,Fantastic Beasts and Where to Find Them,1.0
3667,Percy Jackson & the Olympians: The Lightning T...,1.0
1891,Clash of the Titans,1.0
2008,Journey to the Center of the Earth,1.0
4585,Percy Jackson: Sea of Monsters,1.0
3266,The Chronicles of Narnia: Prince Caspian,1.0
